# Mount Google Drive. 

Make sure you have shortcut to PSA-oak-d-images on the root of your gdrive


In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

# Install Dependencies 

The model is tested on tf 1.15 so, installing it would make sure that it works as expected. Can change this later to NCS2 mode/ TFLite. 

In [2]:
# !pip install tensorflow==1.15

# Import libraries. 

Import necessary libraries. 

In [3]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import imageio
import glob
import ntpath
import time
import os
%matplotlib inline
import argparse
from pathlib import Path
from multiprocessing import Process
from time import time
import pandas as pd
# import imutils
from skimage.transform import resize
from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor(max_workers=16)

# Dummy Argparser

set up a dummy argparser for making changes to input paths and models easily


In [4]:
class Args:
    input_path='D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/'
    model_path='D:/00_NCSU/Fall2020/ECE633_IndividualTopics/OAK-D-Weed-Cam/Model/deeplabv3+/models/3_class_model_mobilenet_v3_small_v2.1/3_class_model_mobilenet_v3_small_v2.1_1080x1920.pb'
    model_size=(1080,1920)

args = Args()
input_path = args.input_path
current_input_size=args.model_size
current_model = args.model_path

In [5]:
import pickle

biomass_availability = [
    '2020_09_23_09_39_44',
    '2020_09_23_09_41_08',
    '2020_09_23_09_42_34',
    '2020_09_23_09_43_53',
    '2020_09_23_09_44_44',
    '2020_09_23_09_45_44',
    '2020_09_23_09_47_05',
    '2020_09_23_09_48_05',
    '2020_09_23_09_48_58',
    '2020_09_23_09_49_56',
    '2020_09_23_09_51_34',
    '2020_09_23_09_52_18',
    '2020_10_02_09_48_46',
    '2020_10_02_09_59_14',
    '2020_10_02_09_56_26',
    '2020_10_02_10_07_35',
    '2020_10_02_10_05_44',
    '2020_10_02_10_06_03',
    '2020_10_02_10_17_15',
    '2020_10_02_10_51_00',
    '2020_10_02_10_52_34',
    '2020_10_02_10_58_15',
    '2020_10_02_11_00_36',
    '2020_10_09_10_22_21',
    '2020_10_09_10_22_57',
    '2020_10_09_10_35_10',
    '2020_10_09_10_35_43',
    '2020_10_09_10_42_33',
    '2020_10_09_10_43_20',
    '2020_10_09_10_52_09',
    '2020_10_09_10_52_40',
    '2020_10_09_10_59_08',
    '2020_10_09_11_00_34',
    '2020_10_09_11_07_13',
    '2020_10_09_11_08_47',
    '2020_10_09_11_15_26',
    '2020_10_09_11_16_44',
    '2020_10_09_11_27_14',
    '2020_10_09_11_27_55',
    '2020_10_09_11_33_58',
    '2020_10_09_11_34_38',
    '2020_10_09_11_40_49',
    '2020_10_09_11_41_47',
    '2020_10_09_11_47_33',
    '2020_10_09_11_48_22'
]

data_dir = 'D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/'

In [6]:
# data = pd.read_csv(input_path+'annotation.txt', sep="\t", header=None)
# data.columns=["Path", "good_bad","labeler"]
# data.shape

# Deeplabv3 model 

class to run the deeplab model inference. Returns semantic probabilities and segmentation map. 

In [7]:
class DeepLabModel():
    """Class to load deeplab model and run inference."""

    INPUT_TENSOR_NAME = 'ImageTensor:0'
    OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
    SOFTMAX_TENSOR_NAME = 'SemanticProbabilities:0'
    INPUT_SIZE = current_input_size


    def __init__(self, path):
        """Creates and loads pretrained deeplab model."""
        self.graph = tf.Graph()

        graph_def = None
        # Extract frozen graph from tar archive.

        with tf.gfile.GFile(path, 'rb')as file_handle:
            graph_def = tf.GraphDef.FromString(file_handle.read())

        if graph_def is None:
            raise RuntimeError('Cannot find inference graph')

        with self.graph.as_default():
            tf.import_graph_def(graph_def, name='')
        
        # To Run on CPU, uncomment below and add config to self.session as: ", config=config"
        config = tf.ConfigProto(
            device_count = {'GPU': 0}
            )

        self.sess = tf.Session(graph=self.graph, config=config)


    def run(self, image):
        """Runs inference on a single image.

        Args:
          image: A PIL.Image object, raw input image.

        Returns:
          seg_map: np.array. values of pixels are classes
        """

        # width, height,ch = image.shape

        # resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
        # target_size = (int(resize_ratio * width), int(resize_ratio * height))

        resized_image = cv2.resize(image, (current_input_size))

        batch_seg_map, batch_prob_map = self.sess.run(
           [self.OUTPUT_TENSOR_NAME,
           self.SOFTMAX_TENSOR_NAME],
           feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(image)]})

        seg_map = batch_seg_map[0]        
        # seg_map = resize(seg_map.astype(np.uint8), (height, width), preserve_range=True, order=0, anti_aliasing=False)
        prob_map = batch_prob_map[0]        
        return seg_map, prob_map

# Disparity color to Depth 


Disparity color is a colorized version of the disparity stream, so to retrieve the actual disparity values from that, we will need to convert it back using the LUT shown below. 

### Two approaches
1. So, using the two LUTs, we would get depth values from disparity_color. 
2. Direct calculation of depth using vectorization


In [8]:
def gen_depth(disparity_color):
    c_disp_to_color =[   (0,0,0), (110,0,0), (120,0,0), (130,0,0),
    (140,0,0), (150,0,0), (160,0,0), (170,0,0), (180,0,0), (190,0,0), (200,0,0),
    (210,0,0), (220,0,0), (230,0,0), (240,0,0), (250,0,0), (255,0,0), (255,10,0),
    (255,20,0), (255,30,0), (255,40,0), (255,50,0), (255,60,0), (255,70,0),
    (255,80,0), (255,90,0), (255,100,0), (255,110,0), (255,120,0), (255,130,0),
    (255,140,0), (255,150,0), (255,160,0), (255,170,0), (255,180,0), (255,190,0),
    (255,200,0), (255,210,0), (255,220,0), (255,230,0), (255,240,0), (255,250,0),
    (255,255,0), (245,255,10), (235,255,20), (225,255,30), (215,255,40),
    (205,255,50), (195,255,60), (185,255,70), (175,255,80), (165,255,90),
    (155,255,100), (145,255,110), (135,255,120), (125,255,130), (115,255,140),
    (105,255,150), (95,255,160), (85,255,170), (75,255,180), (65,255,190),
    (55,255,200), (45,255,210), (35,255,220), (25,255,230), (15,255,240),
    (5,255,250), (0,245,255), (0,235,255), (0,225,255), (0,215,255), (0,205,255),
    (0,195,255), (0,185,255), (0,175,255), (0,165,255), (0,155,255), (0,145,255),
    (0,135,255), (0,125,255), (0,115,255), (0,105,255), (0,95,255), (0,85,255),
    (0,75,255), (0,65,255), (0,55,255), (0,45,255), (0,35,255), (0,25,255),
    (0,15,255), (0,5,255), (0,0,245), (0,0,235), (0,0,225)]
    disparity=np.zeros((720,1280))
    for i in range(len(c_disp_to_color)):
        mask = np.all(disparity_color == c_disp_to_color[i],axis=2)
        #apply the mask to overwrite the pixels
        disparity[ mask ] = i
    depth =  27 * 883.14 * np.reciprocal(disparity)
    depth=np.where(depth==float('inf'), 65535, depth) 
    return depth

# Calculate Volume 

Class to calculate volume. Initialized with the annotations.csv read into a dataframe. The DataFrame is traversed and the labels for good or bad images are checked, if the image is labelled to be good, and it is taken after Oct 9th (has the depth.npy arrays) it is sent through the calculation pipeline for volume calculation. 


In [9]:
test_biomass = ['2020_10_02_10_07_35',
'2020_10_02_10_58_15']

In [294]:
class CalcVolume:    
    model = DeepLabModel(current_model)
    data= pd.DataFrame({'A' : []})
    image_basepath=''
    data_frame=[]
    def __init__(self, input_path):
        self.path = input_path
#         self.data = pd.read_csv(input_path+'annotation.txt', sep="\t", header=None)
#         self.data.columns=["Path", "good_bad","labeler"]
    def get_annotated_directories(self):
        global biomass_availability
        annotatable = []
        for root, dirs, _ in os.walk(self.path):
            for d in dirs:
                if d in biomass_availability:
                    path = os.path.join(root, d)

                    images = [os.path.join(path, f) for f in os.listdir(path) if os.path.isdir(os.path.join(path,f))]
                    # images = [f for f in images if ('depth.npy' in os.listdir(f)) and ('color.png' in os.listdir(f)) and not('box_annotation.pkl') in os.listdir(f)]
                    images = [f for f in images \
                               if (('depth.npy' in os.listdir(f)) or ('disparity_color.png' in os.listdir(f))) and \
                                 ('color.png' in os.listdir(f)) \
                                      and ('box_annotation.pkl') in os.listdir(f)]
                    annotatable.extend(images)
        return (n for n in annotatable)
    
    def get_height_his_dirs(self):
        global biomass_availability
        annotatable = []
        for root, dirs, _ in os.walk(self.path):
            for d in dirs:
                if d in biomass_availability:
                    path = os.path.join(root, d)

                    images = [os.path.join(path, f) for f in os.listdir(path) if os.path.isdir(os.path.join(path,f))]
                    # images = [f for f in images if ('depth.npy' in os.listdir(f)) and ('color.png' in os.listdir(f)) and not('box_annotation.pkl') in os.listdir(f)]
                    images = [f for f in images \
                               if (('depth.npy' in os.listdir(f)) or ('disparity_color.png' in os.listdir(f))) and \
                                 ('color.png' in os.listdir(f)) \
                                      and ('box_annotation.pkl' in os.listdir(f)) and\
                             ('hist_img.pkl' in os.listdir(f))]
                    annotatable.extend(images)
        return (n for n in annotatable)
    
    def detect_white(self, image):
        hsv=cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        # define range of white color in HSV
        lower_white = np.array([0, 0, 222])
        upper_white = np.array([150, 30, 255])
        # lower_white = np.array([0,0,0], dtype=np.uint8)
        # upper_white = np.array([0,0,255], dtype=np.uint8)
        # Threshold the HSV image to get only white colors
        mask = cv2.inRange(hsv, lower_white, upper_white)
        # Bitwise-AND mask and original image
        res = cv2.bitwise_and(image,image, mask= mask)
        return mask, res
    
    def closing(self, rect):
        kernel = np.ones((3,3),np.uint8)
        # kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
        # rect = cv2.morphologyEx(rect, cv2.MORPH_OPEN, kernel, iterations=10)
        h,w=rect.shape[0], rect.shape[1]
        gray = cv2.cvtColor(rect, cv2.COLOR_BGR2GRAY)
        mask1=np.zeros((h,w), np.uint8)    
        cv2.rectangle(mask1,(int(0.25*w),int(0.15*h)),(int(0.75*w),int(0.85*h)),(255,255,255),-1)
        gray=cv2.bitwise_and(gray, gray, mask=mask1)
        b1= cv2.medianBlur(gray, 13)
        c1=cv2.dilate(b1, np.ones((3,3)), iterations = 100)
        e1 = cv2.erode(c1, np.ones((3,3)), iterations = 85)
        return e1
    
    def floodFill(self,im_in):
        h,w=im_in.shape[0],im_in.shape[1]
        kernel=np.ones((3,3), np.uint8)
        # mask1 = np.zeros((h, w), np.uint8)
        im_floodfill = im_in.copy()
        mask = np.zeros((h+2, w+2), np.uint8)
        # Floodfill from point (0, 0)
        cv2.floodFill(im_floodfill, mask, (0,0), (255,255,255));
        # Invert floodfilled image
        im_floodfill_inv = cv2.bitwise_not(im_floodfill)
        im_floodfill=cv2.dilate(im_floodfill, kernel, iterations = 25) 
        mask1=np.zeros((h,w), np.uint8)    
        cv2.rectangle(mask1,(int(0.25*w),int(0.15*h)),(int(0.75*w),int(0.85*h)),(255,255,255),-1)
        im_floodfill_inv=cv2.bitwise_and(im_floodfill_inv, im_floodfill_inv, mask=mask1)
        _,im_floodfill_inv=cv2.threshold(im_floodfill_inv,127,255,cv2.THRESH_BINARY)
        # Combine the two images to get the foreground.
        im_out = im_in | im_floodfill_inv    
        return im_floodfill, im_floodfill_inv, im_out
    
    def vis_depth(self,depth):
        depth=(65535//depth+(0.0001)).astype(np.uint8)
        depth = cv2.applyColorMap(depth, cv2.COLORMAP_HOT)
        plt.imshow(depth)
    
    def est_heights_vol(self,bin_mask, seg_mapr,depth):
        vol=0
        #get mask for seg mask in roi
        mask_seg=cv2.bitwise_and(seg_mapr, bin_mask, mask=None)
        # soil mean heights
        soil_mask = seg_mapr == 0
        soil_dist=depth[soil_mask]
        mean_soil_dist = np.mean(soil_dist[np.where(soil_dist!=65535)])
        
        # broadleaf mean heights
        broadleaf_mask = seg_mapr == 1
        bl_dist=depth[broadleaf_mask]
        mean_bl_dist = np.mean(bl_dist[np.where(bl_dist!=65535)])
        
        broad_leaf_count=np.sum(mask_seg==1)
        # grass mean heights
        grass_mask = seg_mapr == 2
        gr_dist=depth[grass_mask]
        mean_gr_dist = np.mean(gr_dist[np.where(gr_dist!=65535)])
        
        grass_count=np.sum(mask_seg==2)
        #roi area
        area = np.count_nonzero(mask_seg)/(np.count_nonzero(bin_mask))*0.25
        #depths with invalids replaced by mean soil hts. 
        dists=np.where(depth ==65535, mean_soil_dist, depth)

        height_array=np.abs(mean_soil_dist-dists)
        soil_ht_array=height_array.copy()
        bl_ht_array=height_array.copy()
        gr_ht_array=height_array.copy()
        
        soil_ht_array[soil_mask] = 0
        bl_ht_array[broadleaf_mask] = 0
        gr_ht_array[grass_mask] = 0
        
        mean_soil_ht=np.mean(soil_ht_array)
        mean_bl_ht=np.mean(bl_ht_array)
        mean_gr_ht=np.mean(gr_ht_array)
        mean_heights=[mean_soil_ht, mean_bl_ht, mean_gr_ht]       
        height_arrays = [height_array, soil_ht_array, bl_ht_array, gr_ht_array]
        
        ht_roi = height_array.copy().astype(np.uint16)
        ht_roi = cv2.bitwise_and(ht_roi, ht_roi, bin_mask.astype(np.uint16))

        plt.figure(figsize=(10,6))
        n_a, bins_a, _=plt.hist(height_arrays[0].ravel(),bins=np.arange(0,65535,250), alpha=0.25, label='heights_all')
        n_s, bins_s, _=plt.hist(height_arrays[1].ravel(),bins=np.arange(0,65535,250), alpha=0.25, label='heights_soil')
        n_b, bins_b, _=plt.hist(height_arrays[2].ravel(),bins=np.arange(0,65535,250), alpha=0.25, label='heights_broadleaf')
        n_g, bins_g, _=plt.hist(height_arrays[3].ravel(),bins=np.arange(0,65535,250),alpha=0.25, label='heights_grass')
        plt.legend(loc='upper right'); plt.title('histograms for whole image heights'); plt.show()
        
#         with open(os.path.join(self.image_basepath+'/hist_all.pkl'), 'wb') as f:
#                 pickle.dump(np.hstack((n_a, bins_a[1:])), f)
#         with open(os.path.join(self.image_basepath+'/hist_s.pkl'), 'wb') as f:
#                 pickle.dump(np.hstack((n_s, bins_s[1:])), f)
#         with open(os.path.join(self.image_basepath+'/hist_b.pkl'), 'wb') as f:
#                 pickle.dump(np.hstack((n_b, bins_b[1:])), f)
#         with open(os.path.join(self.image_basepath+'/hist_g.pkl'), 'wb') as f:
#                 pickle.dump(np.hstack((n_g, bins_g[1:])), f)
    
        hist_image_data = np.hstack((n_a, n_b, n_s, n_g))
        with open(os.path.join(self.image_basepath+'/hist_img.pkl'), 'wb') as f:
                pickle.dump(hist_image_data, f)
                
#         np.save(self.image_basepath+'/hist_all.npy', np.array([n_a, bins_a[1:]]))
#         np.savetxt(self.image_basepath+'/hist_s.npy', np.array([n_s, bins_s[1:]])) 
#         np.savetxt(self.image_basepath+'/hist_b.npy', np.array([n_b, bins_b[1:]])) 
#         np.savetxt(self.image_basepath+'/hist_g.npy', np.array([n_g, bins_g[1:]]))  

        
        soil_ht_array=cv2.bitwise_and(soil_ht_array.astype(np.uint16), soil_ht_array.astype(np.uint16), bin_mask.astype(np.uint16))
        soil_ht_array=soil_ht_array[soil_ht_array!=0]
        bl_ht_array=cv2.bitwise_and(bl_ht_array.astype(np.uint16), bl_ht_array.astype(np.uint16), bin_mask.astype(np.uint16))
        bl_ht_array=bl_ht_array[bl_ht_array!=0]
        gr_ht_array=cv2.bitwise_and(gr_ht_array.astype(np.uint16), gr_ht_array.astype(np.uint16), bin_mask.astype(np.uint16))
        gr_ht_array=gr_ht_array[gr_ht_array!=0]
        plt.figure(figsize=(10,6))
        
        n_a, bins_a, _=plt.hist(ht_roi[ht_roi!=0].ravel(),bins=np.arange(0,65535,250),alpha=0.25, label='heights_all')
        n_s, bins_s, _=plt.hist(soil_ht_array.ravel(),bins=np.arange(0,65535,250), alpha=0.25, label='heights_soil')
        n_b, bins_b, _=plt.hist(bl_ht_array.ravel(),bins=np.arange(0,65535,250),  alpha=0.25, label='heights_broadleaf')
        n_g, bins_g, _=plt.hist(gr_ht_array.ravel(),bins=np.arange(0,65535,250), alpha=0.25, label='heights_grass')
        plt.legend(loc='upper right'); plt.title('histograms for ROI image heights');  plt.show()
        
#         with open(os.path.join(self.image_basepath+'/hist_all_roi.pkl'), 'wb') as f:
#                 pickle.dump(np.hstack((n_a, bins_a[1:])), f)
#         with open(os.path.join(self.image_basepath+'/hist_s_roi.pkl'), 'wb') as f:
#             pickle.dump(np.hstack((n_s, bins_s[1:])), f)
#         with open(os.path.join(self.image_basepath+'/hist_b_roi.pkl'), 'wb') as f:
#                 pickle.dump(np.hstack((n_b, bins_b[1:])), f)
#         with open(os.path.join(self.image_basepath+'/hist_g_roi.pkl'), 'wb') as f:
#             pickle.dump(np.hstack((n_g, bins_g[1:])), f)
        hist_roi_data = np.hstack((n_a, n_b, n_s, n_g))
        with open(os.path.join(self.image_basepath+'/hist_roi.pkl'), 'wb') as f:
            pickle.dump(hist_image_data, f)
        bins = bins_a
        with open(os.path.join(self.image_basepath+'/bins.pkl'), 'wb') as f:
            pickle.dump(bins, f)
#         np.save(self.image_basepath+'/hist_all_roi.npy', np.array([n_a, bins_a[1:]]).T)  
#         np.save(self.image_basepath+'/hist_s_roi.npy', np.array([n_s, bins_s[1:]]).T)   
#         np.save(self.image_basepath+'/hist_b_roi.npy', np.array([n_b, bins_b[1:]]).T)  
#         np.save(self.image_basepath+'/hist_g_roi.npy', np.array([n_g, bins_g[1:]]).T)   

        vol = np.sum((ht_roi*0.01)/(broad_leaf_count+grass_count)*area)
        
        return mean_heights, height_arrays,vol

    def pipeline(self):
        vols=[]
        heights=[]
        means=[]
        paths=[]
        annotated = self.get_annotated_directories()
        while True:
            try:
                self.image_basepath = next(annotated)
            except StopIteration:
                break
            color_path = os.path.join(self.image_basepath, 'color.png')
            paths.append(color_path)
            depth_path = os.path.join(self.image_basepath, 'depth.npy')
            disparity_path = os.path.join(self.image_basepath, 'disparity_color.png')
            annotation_path = os.path.join(self.image_basepath, 'box_annotation.pkl')
            print(color_path)
            image=cv2.cvtColor(cv2.imread(color_path), cv2.COLOR_BGR2RGB)
            try:
                depth=np.load(depth_path)
            except:
                disparity_color = cv2.imread(disparity_path)
                depth=gen_depth(disparity_color)

            file = open(annotation_path,'rb')
            object_file = pickle.load(file)
            file.close()
            annotations=object_file
            if len(annotations)==0:
                continue
            #visualize original image and depth
            plt.figure(figsize=(10,6))
            plt.imshow(image)
#             plt.figure(figsize=(10,6))
#             self.vis_depth(depth)


            seg_map, prob_map=self.model.run(image)
            seg_mapr=cv2.resize(seg_map.astype(np.uint8),(depth.shape[1],depth.shape[0]), interpolation = cv2.INTER_LANCZOS4)
#             plt.figure(figsize=(10,6))
#             plt.imshow(seg_mapr)
            #generate bin_mask from annotations
            bin_mask = np.zeros((1080,1920))
            bin_mask = cv2.fillConvexPoly(bin_mask, np.array(annotations), 1)
            bin_mask=cv2.resize(bin_mask, (1280, 720)).astype(np.uint8)
#             plt.figure(figsize=(10,6))
#             plt.imshow(bin_mask, cmap='gray') 
            #estimate the volume and heights using seg masks. 
            mean_heights, height_arrays,vol=self.est_heights_vol(bin_mask,seg_mapr,depth)
            print(vol)
            vols.append(vol)
            heights.append(height_arrays[0])
            means.append(mean_heights)
        dat=[paths,heights,means,vols]
#         self.data = pd.DataFrame(dat, columns =['Paths', 'Heights', 'Means', 'Vols']) 
        return dat
    def make_path_csv(self):
        path_data=[]
#         hist_all_series=[]
#         hist_all_roi_series=[]
#         hist_s_series=[]
#         hist_s_roi_series=[]
#         hist_b_series=[]
#         hist_b_roi_series=[]
#         hist_g_series=[]
#         hist_g_roi_series=[]
#         hist_img_data=[]
#         hist_roi_data=[]
        hist_img_data_list=[]
        hist_roi_data_list=[]
        bins_data_list=[]
        height_hist_paths = self.get_height_his_dirs()
        while True:
            try:
                self.image_basepath = next(height_hist_paths)
                print(self.image_basepath)
            except StopIteration:
                break
            
#             hist_all=self.image_basepath+'/hist_all.pkl'
#             hist_all_roi=self.image_basepath+'/hist_all_roi.pkl'
#             hist_s=self.image_basepath+'/hist_s.pkl'
#             hist_s_roi=self.image_basepath+'/hist_s_roi.pkl'
#             hist_b=self.image_basepath+'/hist_b.pkl'
#             hist_b_roi=self.image_basepath+'/hist_b_roi.pkl'
#             hist_g=self.image_basepath+'/hist_g.pkl'
#             hist_g_roi=self.image_basepath+'/hist_g_roi.pkl'
#             file = open(hist_all,'rb')
#             hist_all_series.append(pickle.load(file))
#             file.close()
#             file = open(hist_all_roi,'rb')
#             hist_all_roi_series.append(pickle.load(file))
#             file.close()
#             file = open(hist_s,'rb')
#             hist_s_series.append(pickle.load(file))
#             file.close()
#             file = open(hist_s_roi,'rb')
#             hist_s_roi_series.append(pickle.load(file))
#             file.close()
#             file = open(hist_b,'rb')
#             hist_b_series.append(pickle.load(file))
#             file.close()
#             file = open(hist_b_roi,'rb')
#             hist_b_roi_series.append(pickle.load(file))
#             file.close()
#             file = open(hist_g,'rb')
#             hist_g_series.append(pickle.load(file))
#             file.close()
#             file = open(hist_g_roi,'rb')
#             hist_g_roi_series.append(pickle.load(file))
#             file.close()
            hist_img=self.image_basepath+'/hist_img.pkl'
            hist_roi=self.image_basepath+'/hist_roi.pkl'
            bins=self.image_basepath+'/bins.pkl'
            file = open(hist_img,'rb')
            hist_img_data = pickle.load(file)
            file.close()
            file = open(hist_roi,'rb')
            hist_roi_data = pickle.load(file)
            file.close()
            file = open(bins,'rb')
            bins_data = pickle.load(file)
            file.close()
            
            hist_img_data_list.append(hist_img_data)
            hist_roi_data_list.append(hist_roi_data)
            bins_data_list.append(bins_data)

#             hist_all_series.append(np.load(hist_all, allow_pickle=True))
#             hist_all_roi_series.append(np.load(hist_all_roi,allow_pickle=True))
#             hist_s_series.append(np.load(hist_s,allow_pickle=True))
#             hist_s_roi_series.append(np.load(hist_s_roi,allow_pickle=True))
#             hist_b_series.append(np.load(hist_b,allow_pickle=True))
#             hist_b_roi_series.append(np.load(hist_b_roi,allow_pickle=True))
#             hist_g_series.append(np.load(hist_g,allow_pickle=True))
#             hist_g_roi_series.append(np.load(hist_g_roi,allow_pickle=True))
            path_data.append(self.image_basepath)
#         return [path_data, hist_all_series, hist_all_roi_series, hist_s_series, hist_s_roi_series\
#                        ,hist_b_series, hist_b_roi_series, hist_g_series, hist_g_roi_series]
        return [path_data, hist_img_data_list,hist_roi_data_list,bins_data_list]
                

In [ ]:
calc_vol=CalcVolume(input_path)
dat=calc_vol.pipeline()
# calc_vol.data.to_csv(args.input_path+'features.csv')

In [297]:
len(dat[0]), len(dat[1]), len(dat[2]), len(dat[3])


(659, 648, 648, 648)

In [298]:
# df = pd.DataFrame({'Path': dat[0],
#                    'height_array':dat[1],
#                   'mean_heights':dat[2],
#                   'est_vols':dat[3]})

In [299]:
# plt.hist(height_arrays[0].ravel(), 96); 
# height_arrays[0], mean_heights
# calc_vol.data=pd.DataFrame(dat, cols = ['paths', 'raw_heights', 'means', 'vols'])
# calc_vol.data.head()

In [377]:
calc_vol.data_frame = calc_vol.make_path_csv()

D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\2020_09_23_09_39_44\037c5453-56e1-42b5-b2a2-b7b4204d1c4e
D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\2020_09_23_09_39_44\03a825f1-8954-4dff-91e9-c15753d2ce7a
D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\2020_09_23_09_39_44\11b462da-50d3-4206-82d0-b230b086d11c
D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\2020_09_23_09_39_44\167b676f-f7af-4f31-bf45-dd1700e54a94
D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\2020_09_23_09_39_44\192a849d-cbf0-4f00-b93f-56822bd9bf51
D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\2020_09_23_09_39_44\1d37b642-9741-4ea0-b0b8-dea57a0b208b
D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\2020_09_23_09_39_44\28a799dc-4f7e-4d36-8656-3e9dd1c75832
D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\2020_09_23_09_39_44\2d7de2a6-2464-4f8f-8a7e-9de4d2b57200
D:/00_NC

D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\2020_09_23_09_49_56\e8b0877d-8521-41dd-9e6c-0f377432972c
D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\2020_09_23_09_49_56\f08c2a78-1a0f-42bd-8a22-c6bad7a0d4e8
D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\2020_09_23_09_51_34\096eb0ad-371d-46f4-9768-c9f19d91be42
D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\2020_09_23_09_51_34\10eddad8-5ceb-4a86-af63-56ae224b8d33
D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\2020_09_23_09_51_34\265a44a3-9505-4300-b3cb-a39ad1fecc0f
D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\2020_09_23_09_51_34\2d26cd79-7607-45f1-bdc2-dbb1afabaf34
D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\2020_09_23_09_51_34\303184c6-47aa-42ec-9325-6c6f3f129948
D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\2020_09_23_09_51_34\36df7eaf-f359-43e1-bf11-2deaf58bb2da
D:/00_NC

In [378]:
calc_vol.data_frame[1][1]

array([620901., 218840.,  49183., ...,      0.,      0.,      0.])

In [405]:
dframe = np.array(np.hstack((calc_vol.data_frame[1],calc_vol.data_frame[2],calc_vol.data_frame[3])))
pframe=np.array(calc_vol.data_frame[0])

d
# df = pd.DataFrame({'Path': calc_vol.data_frame[0],
                   
#                    'freqs_img':calc_vol.data_frame[1],
#                   'freqs_roi':calc_vol.data_frame[2],
#                   'bins':calc_vol.data_frame[3]})
df=df.T
# df.columns=['Path', 'freqs_img', 'freqs_roi', 'bins']
# df.head()
pframe

array(['D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\\2020_09_23_09_39_44\\037c5453-56e1-42b5-b2a2-b7b4204d1c4e',
       'D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\\2020_09_23_09_39_44\\03a825f1-8954-4dff-91e9-c15753d2ce7a',
       'D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\\2020_09_23_09_39_44\\11b462da-50d3-4206-82d0-b230b086d11c',
       'D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\\2020_09_23_09_39_44\\167b676f-f7af-4f31-bf45-dd1700e54a94',
       'D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\\2020_09_23_09_39_44\\192a849d-cbf0-4f00-b93f-56822bd9bf51',
       'D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\\2020_09_23_09_39_44\\1d37b642-9741-4ea0-b0b8-dea57a0b208b',
       'D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/NC_09_23_2020\\2020_09_23_09_39_44\\28a799dc-4f7e-4d36-8656-3e9dd1c75832',
       'D:/00_NCSU/00_Resources/00_Datasets/PSA-

In [396]:
np.hstack((calc_vol.data_frame[1],calc_vol.data_frame[2],calc_vol.data_frame[3])).shape

(648, 2359)

In [375]:
df.to_csv(args.input_path+'test_feats.csv')

In [355]:
df = pd.read_csv('D:/00_NCSU/00_Resources/00_Datasets/PSA-oak-d-images/test_feats.csv', dtype={'Path':str,
                                                                                              'freqs_img':np.float64,
                                                                                              'freqs_roi':np.float64,
                                                                                              'bins':np.int32})
df.head()

ValueError: could not convert string to float: '[608472. 212537.  65851. ...      0.      0.      0.]'

In [316]:
df['freqs_roi'][0]

'[608472. 212537.  65851. ...      0.      0.      0.]'

In [147]:
np.loadtxt(a)

ValueError: could not convert string to float: '[array([[3.25630000e+05,'

In [144]:
a.split('\n')

['[array([[3.25630000e+05, 2.45780000e+04, 1.02116000e+05, 9.76350000e+04,',
 '        7.09420000e+04, 9.50970000e+04, 3.69560000e+04, 3.16830000e+04,',
 '        4.14100000e+04, 2.43680000e+04, 1.22220000e+04, 1.27400000e+04,',
 '        7.76600000e+03, 5.25100000e+03, 1.08100000e+03, 7.17000000e+02,',
 '        3.73200000e+03, 7.06000000e+02, 7.36000000e+02, 4.79000000e+02,',
 '        2.86300000e+03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,',
 '        0.00000000e+00, 2.60000000e+03, 0.00000000e+00, 0.00000000e+00,',
 '        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,',
 '        2.41600000e+03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,',
 '        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,',
 '        2.11000000e+03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,',
 '        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,',
 '        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.12100000e+03,',
 '        0

In [132]:
a[:len(a)//2]

['[array([[3.25630000e+05,',
 '2.45780000e+04,',
 '1.02116000e+05,',
 '9.76350000e+04,',
 '7.09420000e+04,',
 '9.50970000e+04,',
 '3.69560000e+04,',
 '3.16830000e+04,',
 '4.14100000e+04,',
 '2.43680000e+04,',
 '1.22220000e+04,',
 '1.27400000e+04,',
 '7.76600000e+03,',
 '5.25100000e+03,',
 '1.08100000e+03,',
 '7.17000000e+02,',
 '3.73200000e+03,',
 '7.06000000e+02,',
 '7.36000000e+02,',
 '4.79000000e+02,',
 '2.86300000e+03,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '2.60000000e+03,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '2.41600000e+03,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '2.11000000e+03,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.00000000e+00,',
 '0.0000000

# Scratch Cells

Testing out improvements

In [ ]:
import pickle
images = []
depths = []

for i in range(len(calc_vol.path)):
#     print(os.listdir(calc_vol.path[i]))
    images.append(cv2.cvtColor(cv2.imread(calc_vol.path[i]+'color.png'), cv2.COLOR_BGR2RGB))
#     plt.imshow(im)
    depths.append(np.load(calc_vol.path[i]+'/depth.npy'))
    file = open(calc_vol.path[i]+'box_annotation.pkl','rb')
    object_file = pickle.load(file)
    file.close()
    annotations.append(object_file)